In [1]:
#STEP1: Generate JSON FILE
import json

def generate_json_file(objective, normalization, pols_array, json_file_path):
    damped_rational_base = "0.3678794411"
    damped_rational_constant = "1"
    poles = []
    pmp_data = {
        "objective": objective,
        "normalization": normalization,
        "PositiveMatrixWithPrefactorArray":
        [   {"DampedRational": {
        "base": damped_rational_base,
        "constant": damped_rational_constant,
        "poles": poles},
        "polynomials": polynomials
            } for polynomials in pols_array] }
    
    with open(json_file_path, 'w') as f:
        json.dump(pmp_data, f, indent=4)

    print(f"JSON file saved at {json_file_path}")

# Example usage
objective_vector = ["0", "-1"]
normalization_vector = ["1", "0"]

# Example array of polynomials
pols_array = [
    [[[["1","0","0","0","1"],["0","0","1","0","0.08333333333"]]]]
]

# Construct the matrices for the JSON
json_file_path = "/Users/faizaan/OneDrive - Indian Institute of Science/Python_Wrapper_For_SDPB/Trial_NBs/pmp_input.json"

generate_json_file(objective_vector, normalization_vector, pols_array, json_file_path)


#RUN SDPB PIPELINE
import subprocess
import os
import shutil

# Define the paths
base_path = "/Users/faizaan/OneDrive - Indian Institute of Science/Python_Wrapper_For_SDPB/Trial_NBs"
input_json = "pmp_input.json"
sdp_dir = "sdp"  # Just the directory name
output_path = "sdp_out"

#Step 0: Delete previous directories
directories_to_delete = [sdp_dir, output_path, "sdp.ck"]
for directory in directories_to_delete:
    dir_path = os.path.join(base_path, directory)
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)  # Recursively delete the directory

# Step 1: Run pmp2sdp
pmp2sdp_command = [
    "docker", "run", "-v", f"{base_path}:/usr/local/share/sdpb",
    "bootstrapcollaboration/sdpb:master", "mpirun", "--allow-run-as-root", "-n", "4",
    "pmp2sdp", "--precision", "1024",
    "-i", f"/usr/local/share/sdpb/{input_json}",
    "-o", f"/usr/local/share/sdpb/{sdp_dir}"
]

def run_pmp2sdp(base_path, input_json, sdp_dir, process_thread_array):
    p, q = process_thread_array  # Unpack processes and threads
    pmp2sdp_command = [
        "docker", "run", "-v", f"{base_path}:/usr/local/share/sdpb",
        "bootstrapcollaboration/sdpb:master", 
        "mpirun", "--allow-run-as-root", "-n", str(p),  # Use p processes
        "pmp2sdp", "--precision", "1024",
        "-i", f"/usr/local/share/sdpb/{input_json}",
        "-o", f"/usr/local/share/sdpb/{sdp_dir}"
    ]
    # Set number of threads per process using environment variable
    subprocess.run(pmp2sdp_command, env={"OMP_NUM_THREADS": str(q)})














# Step 2: Run SDPB
sdpb_command = [
    "docker", "run", "-v", f"{base_path}:/usr/local/share/sdpb",
    "bootstrapcollaboration/sdpb:master", "mpirun", "--allow-run-as-root", "-n", "4",
    "sdpb", "--precision=1024",
    "-s", f"/usr/local/share/sdpb/{sdp_dir}"
]

# Execute the commands
subprocess.run(pmp2sdp_command)
subprocess.run(sdpb_command)


#PRINT OUTPUT
import re
import numpy as np

# Path to the SDPB out.txt file and y.txt file for the primal objective and solution respectively 
output_file_path = "/Users/faizaan/OneDrive - Indian Institute of Science/Python_Wrapper_For_SDPB/Trial_NBs/sdp_out/out.txt"
solution_file_path = "/Users/faizaan/OneDrive - Indian Institute of Science/Python_Wrapper_For_SDPB/Trial_NBs/sdp_out/y.txt"


# Read and store primal objective
pObj = None
with open(output_file_path, 'r') as f:
    content = f.read()
    match = re.search(r"primalObjective = ([\-\d\.e]+);", content)
    if match:
        pObj = np.float64(match.group(1))  # Store the value as float64
        
# Read and store olution
with open(solution_file_path, 'r') as f:
    # Read all lines, skipping the first one
    lines = f.readlines()[1:]
    pSol = np.array([np.float64(line.strip()) for line in lines])

print(f"pObj:{pObj}")
print(f"pSol: {pSol}")
pObj, pSol

JSON file saved at /Users/faizaan/OneDrive - Indian Institute of Science/Python_Wrapper_For_SDPB/Trial_NBs/pmp_input.json
Processed 1 SDP blocks in 0.188 seconds, output: /usr/local/share/sdpb/sdp
2024-Oct-31 19:45:00 Start SDPB
SDPB version: 3.0.0-76-g693d9e05
MPI processes: 4, nodes: 1
SDP directory   : "/usr/local/share/sdpb/sdp"
out directory   : "/usr/local/share/sdpb/sdp_out"

Parameters:
maxIterations                = 500
maxRuntime                   = 9223372036854775807
checkpointInterval           = 3600
maxSharedMemory              = 0 B
findPrimalFeasible           = false
findDualFeasible             = false
detectPrimalFeasibleJump     = false
detectDualFeasibleJump       = false
precision(actual)            = 1024(1024)
dualityGapThreshold          = 1e-30
primalErrorThreshold         = 1e-30
dualErrorThreshold           = 1e-30
initialMatrixScalePrimal     = 1e+20
initialMatrixScaleDual       = 1e+20
feasibleCenteringParameter   = 0.1
infeasibleCenteringParameter = 0.3


	MemTotal: 3.8 GB
	Required memory estimate (excluding shared memory windows): 525.6 MB
	To prevent OOM, SDPB will set --maxSharedMemory to 50% of the remaining memory, i.e. 1.7 GB
	In case of OOM, consider increasing number of nodes and/or decreasing --maxSharedMemory limit.


1            0 1.0e+40  +0.00       +0.00       0.00       +1.00e+20   +1.00       +1.09e+20   0.617    0.651    0.300
2            0 5.1e+39  +1.97e+20   -1.58e+20   1.00       +3.83e+19   +0.383      +3.81e+19   0.652    0.638    0.300
3            0 2.5e+39  +2.84e+20   -3.05e+20   1.00       +1.33e+19   +0.133      +1.38e+19   0.649    0.639    0.300
4            0 1.2e+39  +3.89e+20   -4.62e+20   1.00       +4.68e+18   +0.0468     +4.98e+18   0.642    0.637    0.300
5            0 6.1e+38  +5.27e+20   -6.59e+20   1.00       +1.67e+18   +0.0167     +1.81e+18   0.640    0.635    0.300
6            0 3.0e+38  +7.13e+20   -9.25e+20   1.00       +6.04e+17   +0.00604    +6.59e+17   0.637    0.634    0.300
7            0 1.5e+38  +9.67e+20   -1.29e+21   1.00       +2.19e+17   +0.00219    +2.41e+17   0.636    0.633    0.300
8            0 7.7e+37  +1.32e+21   -1.79e+21   1.00       +7.98e+16   +0.000798   +8.84e+16   0.634    0.633    0.300
9            0 3.9e+37  +1.81e+21   -2.48e+21   

(np.float64(1.8402657631381623), array([-1.84026576]))

In [2]:
#Example with 2x2 matrices

import json

# Define your PMP data structure
pmp_data ={
	"objective":[
		"0",
		"-1"
	],
	"normalization":[
		"1",
		"0"
	],
	"PositiveMatrixWithPrefactorArray":[
		{
			"prefactor":{
				"base":"0.3678794412",
				"constant":"1",
				"poles":[]
			},
			"polynomials":[
				[[["1","0","0","0","1"],["1","0","1","0","0.08333333333"]],[["0","0","1"],["0","0.2000000000"]]],
				[[["0","0","1"],["0","0.2000000000"	]],[["2","0","0","0","1"],["0","0","2","0","0.3333333333"]]]
			]
		},
		{
			"prefactor":{
				"base":"0.3678794412",
				"constant":"1",
				"poles":[]
			},
			"polynomials":[
				[[["1","0","0","0","0.7500000000"],["1","0","1","0","0.08333333333"]],[["0","0","1"],["0.5000000000","0.2000000000"]]],
				[[["0","0","1"],["0.5000000000","0.2000000000"]],[["2","0","0","0","0.6000000000"],["0","0","2","0","0.3333333333"]]]
                		  ]
		}
	]
}
# Save the data to a JSON file
json_file_path = "pmp_input.json"
with open(json_file_path, 'w') as f:
    json.dump(pmp_data, f, indent=4)

print(f"JSON file saved at {json_file_path}")


#RUN SDPB PIPELINE
import subprocess
import os
import shutil

# Define the paths
base_path = "/Users/faizaan/OneDrive - Indian Institute of Science/Python_Wrapper_For_SDPB/Trial_NBs"
input_json = "pmp_input.json"
sdp_dir = "sdp"  # Just the directory name
output_path = "sdp_out"

#Step 0: Delete previous directories
directories_to_delete = [sdp_dir, output_path, "sdp.ck"]
for directory in directories_to_delete:
    dir_path = os.path.join(base_path, directory)
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)  # Recursively delete the directory

# Step 1: Run pmp2sdp
pmp2sdp_command = [
    "docker", "run", "-v", f"{base_path}:/usr/local/share/sdpb",
    "bootstrapcollaboration/sdpb:master", "mpirun", "--allow-run-as-root", "-n", "4",
    "pmp2sdp", "--precision", "1024",
    "-i", f"/usr/local/share/sdpb/{input_json}",
    "-o", f"/usr/local/share/sdpb/{sdp_dir}"
]

# Step 2: Run SDPB
sdpb_command = [
    "docker", "run", "-v", f"{base_path}:/usr/local/share/sdpb",
    "bootstrapcollaboration/sdpb:master", "mpirun", "--allow-run-as-root", "-n", "4",
    "sdpb", "--precision=1024",
    "-s", f"/usr/local/share/sdpb/{sdp_dir}"
]

# Execute the commands
subprocess.run(pmp2sdp_command)
subprocess.run(sdpb_command)


#PRINT OUTPUT
import re
import numpy as np

# Path to the SDPB out.txt file and y.txt file for the primal objective and solution respectively 
output_file_path = "/Users/faizaan/OneDrive - Indian Institute of Science/Python_Wrapper_For_SDPB/Trial_NBs/sdp_out/out.txt"
solution_file_path = "/Users/faizaan/OneDrive - Indian Institute of Science/Python_Wrapper_For_SDPB/Trial_NBs/sdp_out/y.txt"


# Read and store primal objective
pObj = None
with open(output_file_path, 'r') as f:
    content = f.read()
    match = re.search(r"primalObjective = ([\-\d\.e]+);", content)
    if match:
        pObj = np.float64(match.group(1))  # Store the value as float64
        
# Read and store olution
with open(solution_file_path, 'r') as f:
    # Read all lines, skipping the first one
    lines = f.readlines()[1:]
    pSol = np.array([np.float64(line.strip()) for line in lines])

print(f"pObj:{pObj}")
print(f"pSol: {pSol}")
pObj, pSol

JSON file saved at pmp_input.json
Processed 2 SDP blocks in 0.372 seconds, output: /usr/local/share/sdpb/sdp
2024-Oct-31 19:45:21 Start SDPB
SDPB version: 3.0.0-76-g693d9e05
MPI processes: 4, nodes: 1
SDP directory   : "/usr/local/share/sdpb/sdp"
out directory   : "/usr/local/share/sdpb/sdp_out"

Parameters:
maxIterations                = 500
maxRuntime                   = 9223372036854775807
checkpointInterval           = 3600
maxSharedMemory              = 0 B
findPrimalFeasible           = false
findDualFeasible             = false
detectPrimalFeasibleJump     = false
detectDualFeasibleJump       = false
precision(actual)            = 1024(1024)
dualityGapThreshold          = 1e-30
primalErrorThreshold         = 1e-30
dualErrorThreshold           = 1e-30
initialMatrixScalePrimal     = 1e+20
initialMatrixScaleDual       = 1e+20
feasibleCenteringParameter   = 0.1
infeasibleCenteringParameter = 0.3
stepLengthReduction          = 0.7
maxComplementarity           = 1e+100
initialCheckpoi

	MemTotal: 3.8 GB
	Required memory estimate (excluding shared memory windows): 510.2 MB
	To prevent OOM, SDPB will set --maxSharedMemory to 50% of the remaining memory, i.e. 1.7 GB
	In case of OOM, consider increasing number of nodes and/or decreasing --maxSharedMemory limit.


2            0 5.7e+39  +1.03e+21   -6.38e+19   1.00       +4.05e+19   +0.405      +5.19e+19   0.823    0.384    0.300
3            0 3.1e+39  +2.69e+21   -9.19e+19   1.00       +7.17e+18   +0.0717     +3.20e+19   1.00     0.665    0.300
4            0 1.7e+39  +5.82e+21   -2.90e+19   1.00       +6.34e-289  +1.26e-287  +1.07e+19   1.00     0.596    0.300
5            0 1.1e+39  +9.31e+21   -1.38e+19   1.00       +5.68e-289  +1.51e-287  +4.33e+18   1.00     0.606    0.300
6            0 6.8e+38  +1.49e+22   -6.02e+18   1.00       +5.74e-289  +2.00e-287  +1.71e+18   1.00     0.618    0.300
7            0 4.2e+38  +2.38e+22   -2.45e+18   1.00       +6.16e-289  +4.25e-287  +6.52e+17   1.00     0.632    0.300
8            1 2.5e+38  +3.82e+22   -9.40e+17   1.00       +6.16e-289  +1.30e-286  +2.40e+17   1.00     0.646    0.300
9            1 1.4e+38  +6.11e+22   -3.41e+17   1.00       +6.16e-289  +1.91e-286  +8.50e+16   1.00     0.661    0.300
10           1 7.5e+37  +9.77e+22   -1.18e+17   

(np.float64(0.6012421507039243), array([-0.60124215]))

In [4]:
#My example/Linear Programming
import json

# Define your PMP data structure
pmp_data = {"objective": [
"0",
"-3",
"-1",
"-2",
"-4"
],
"normalization": [
"1",
"0",
"0",
"0",
"0"
],
"PositiveMatrixWithPrefactorArray": [
{
"polynomials": [
                [[["0"],["1"],["0"],["0"],["0"]]]
                ]
},
{
"polynomials": [
                [[["0"],["0"],["1"],["0"],["0"]]]
                ]
},
{
"polynomials": [
                [[["0"],["0"],["0"],["1"],["0"]]]
                ]
},
{
"polynomials": [
                [[["0"],["0"],["0"],["0"],["1"]]]
                ]
},
{
"polynomials": [
                [[["-5"],["1"],["1"],["1"],["1"]]]
                ]
},
{
"polynomials": [
                [[["-8"],["2"],["1"],["1"],["1"]]]
               ]
},
{
"polynomials": [
                [[["-3"],["-1"],["1"],["2"],["1"]]]
               ]
},
{
"polynomials": [
                [[["-4"],["1"],["3"],["-1"],["1"]]]
               ]
},
{
"polynomials": [
                [[["-2"],["2"],["1"],["1"],["-1"]]]
               ]
}
]
}

# Save the data to a JSON file
json_file_path = "/Users/faizaan/OneDrive - Indian Institute of Science/Python_Wrapper_For_SDPB/Trial_NBs/pmp_input.json"
with open(json_file_path, 'w') as f:
    json.dump(pmp_data, f, indent=4)

print(f"JSON file saved at {json_file_path}")



#RUN SDPB PIPELINE
import subprocess
import os
import shutil

# Define the paths
base_path = "/Users/faizaan/OneDrive - Indian Institute of Science/Python_Wrapper_For_SDPB/Trial_NBs"
input_json = "pmp_input.json"
sdp_dir = "sdp"  # Just the directory name
output_path = "sdp_out"

#Step 0: Delete previous directories
directories_to_delete = [sdp_dir, output_path, "sdp.ck"]
for directory in directories_to_delete:
    dir_path = os.path.join(base_path, directory)
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)  # Recursively delete the directory

# Step 1: Run pmp2sdp
pmp2sdp_command = [
    "docker", "run", "-v", f"{base_path}:/usr/local/share/sdpb",
    "bootstrapcollaboration/sdpb:master", "mpirun", "--allow-run-as-root", "-n", "4",
    "pmp2sdp", "--precision", "1024",
    "-i", f"/usr/local/share/sdpb/{input_json}",
    "-o", f"/usr/local/share/sdpb/{sdp_dir}"
]

# Step 2: Run SDPB
sdpb_command = [
    "docker", "run", "-v", f"{base_path}:/usr/local/share/sdpb",
    "bootstrapcollaboration/sdpb:master", "mpirun", "--allow-run-as-root", "-n", "4",
    "sdpb", "--precision=1024",
    "-s", f"/usr/local/share/sdpb/{sdp_dir}"
]

# Execute the commands
subprocess.run(pmp2sdp_command)
subprocess.run(sdpb_command)


#PRINT OUTPUT
import re
import numpy as np

# Path to the SDPB out.txt file and y.txt file for the primal objective and solution respectively 
output_file_path = "/Users/faizaan/OneDrive - Indian Institute of Science/Python_Wrapper_For_SDPB/Trial_NBs/sdp_out/out.txt"
solution_file_path = "/Users/faizaan/OneDrive - Indian Institute of Science/Python_Wrapper_For_SDPB/Trial_NBs/sdp_out/y.txt"


# Read and store primal objective
pObj = None
with open(output_file_path, 'r') as f:
    content = f.read()
    match = re.search(r"primalObjective = ([\-\d\.e]+);", content)
    if match:
        pObj = np.float64(match.group(1))  # Store the value as float64
        
# Read and store olution
with open(solution_file_path, 'r') as f:
    # Read all lines, skipping the first one
    lines = f.readlines()[1:]
    pSol = np.array([np.float64(line.strip()) for line in lines])

print(f"pObj:{pObj}")
print(f"pSol: {pSol}")
pObj, pSol

JSON file saved at /Users/faizaan/OneDrive - Indian Institute of Science/Python_Wrapper_For_SDPB/Trial_NBs/pmp_input.json
Processed 9 SDP blocks in 0.358 seconds, output: /usr/local/share/sdpb/sdp
2024-Oct-31 19:46:59 Start SDPB
SDPB version: 3.0.0-76-g693d9e05
MPI processes: 4, nodes: 1
SDP directory   : "/usr/local/share/sdpb/sdp"
out directory   : "/usr/local/share/sdpb/sdp_out"

Parameters:
maxIterations                = 500
maxRuntime                   = 9223372036854775807
checkpointInterval           = 3600
maxSharedMemory              = 0 B
findPrimalFeasible           = false
findDualFeasible             = false
detectPrimalFeasibleJump     = false
detectDualFeasibleJump       = false
precision(actual)            = 1024(1024)
dualityGapThreshold          = 1e-30
primalErrorThreshold         = 1e-30
dualErrorThreshold           = 1e-30
initialMatrixScalePrimal     = 1e+20
initialMatrixScaleDual       = 1e+20
feasibleCenteringParameter   = 0.1
infeasibleCenteringParameter = 0.3


	MemTotal: 3.8 GB
	Required memory estimate (excluding shared memory windows): 528.1 MB
	To prevent OOM, SDPB will set --maxSharedMemory to 50% of the remaining memory, i.e. 1.7 GB
	In case of OOM, consider increasing number of nodes and/or decreasing --maxSharedMemory limit.


4            0 1.8e+39  +3.51e+19   -1.46e+21   1.00       +5.27e+18   +0.211      +1.74e-288  0.679    1.00     0.300
5            0 9.4e+38  +1.22e+19   -2.33e+21   1.00       +1.69e+18   +0.0677     +2.46e-288  0.687    1.00     0.300
6            0 4.7e+38  +3.96e+18   -3.72e+21   1.00       +5.29e+17   +0.0211     +2.05e-288  0.693    1.00     0.300
7            0 2.3e+38  +1.24e+18   -5.95e+21   1.00       +1.62e+17   +0.00649    +2.05e-289  0.696    1.00     0.300
8            0 1.1e+38  +3.83e+17   -9.51e+21   1.00       +4.93e+16   +0.00197    +2.05e-289  0.698    1.00     0.300
9            0 5.4e+37  +1.16e+17   -1.52e+22   1.00       +1.49e+16   +0.000596   +1.03e-289  0.699    1.00     0.300
10           0 2.6e+37  +3.51e+16   -2.43e+22   1.00       +4.49e+15   +0.000179   +3.08e-289  0.699    1.00     0.300
11           0 1.3e+37  +1.06e+16   -3.89e+22   1.00       +1.35e+15   +5.39e-05   +1.03e-289  0.700    1.00     0.300
12           0 6.1e+36  +3.18e+15   -6.22e+22   

(np.float64(-8.0),
 array([1.51363314e-30, 8.00000000e+00, 1.51363316e-30, 5.04544388e-31]))